<a href="https://colab.research.google.com/github/illusoryTwin/InnoML/blob/main/ML_task1_2304.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp drive/MyDrive/ML/Assignment2/task_1_train_data.json task_1_train_data.json
!cp drive/MyDrive/ML/Assignment2/task_1_verify.json task_1_verify.json

In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import sklearn

In [151]:
data = pd.read_json('task_1_train_data.json')
waldo = pd.read_json('task_1_verify.json')

data.head()

,browser,os,locale,user_id,location,sites,time,date
0,Chrome,Debian,ur-PK,116,Russia/Moscow,"[{'site': 'bing.net', 'length': 52}, {'site': ...",04:12:00,2017-06-29
1,Firefox,Windows 8,uk-UA,155,France/Paris,"[{'site': 'yahoo.com', 'length': 46}, {'site':...",03:57:00,2016-03-23
2,Safari,MacOS,fr-FR,39,Japan/Tokyo,"[{'site': 'oracle.com', 'length': 335}]",05:26:00,2016-11-17
3,Chrome,Windows 8,nl-NL,175,Australia/Sydney,"[{'site': 'mail.google.com', 'length': 192}, {...",00:05:00,2016-08-23
4,Firefox,Ubuntu,ro-RO,50,USA/San Francisco,"[{'site': 'mail.google.com', 'length': 266}, {...",22:55:00,2016-07-23


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   browser   40000 non-null  object        
 1   os        40000 non-null  object        
 2   locale    40000 non-null  object        
 3   user_id   40000 non-null  int64         
 4   location  40000 non-null  object        
 5   sites     40000 non-null  object        
 6   time      40000 non-null  object        
 7   date      40000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 2.4+ MB


In [152]:
data['target'] = (data.user_id == 0).apply(int)
data['target'].value_counts()

target
0    39800
1      200
Name: count, dtype: int64

In [153]:
def get_utc_timezone(location):
    timezone_mapping = {
        'Russia/Moscow': 'Europe/Moscow',
        'France/Paris': 'Europe/Paris',
        'Japan/Tokyo': 'Asia/Tokyo',
        'Australia/Sydney': 'Australia/Sydney',
        'USA/San Francisco': 'America/Los_Angeles',
        'Canada/Vancouver': 'America/Vancouver',
        'UK/London': 'Europe/London',
        'Canada/Toronto': 'America/Toronto',
        'Malaysia/Kuala Lumpur': 'Asia/Kuala_Lumpur',
        'USA/Miami': 'America/New_York',
        'Italy/Rome': 'Europe/Rome',
        'Spain/Madrid': 'Europe/Madrid',
        'USA/Chicago': 'America/Chicago',
        'Germany/Berlin': 'Europe/Berlin',
        'China/Shanghai': 'Asia/Shanghai',
        'India/Delhi': 'Asia/Kolkata',
        'Brazil/Rio de Janeiro': 'America/Sao_Paulo',
        'Singapore/Singapore': 'Asia/Singapore',
        'Netherlands/Amsterdam': 'Europe/Amsterdam',
        'USA/New York': 'America/New_York',
        'New Zealand/Auckland': 'Pacific/Auckland'
    }
    return timezone_mapping.get(location, None)

data['location_for_utc'] = data['location'].apply(get_utc_timezone)
data

,browser,os,locale,user_id,location,sites,time,date,target,location_for_utc
0,Chrome,Debian,ur-PK,116,Russia/Moscow,"[{'site': 'bing.net', 'length': 52}, {'site': ...",04:12:00,2017-06-29,0,Europe/Moscow
1,Firefox,Windows 8,uk-UA,155,France/Paris,"[{'site': 'yahoo.com', 'length': 46}, {'site':...",03:57:00,2016-03-23,0,Europe/Paris
2,Safari,MacOS,fr-FR,39,Japan/Tokyo,"[{'site': 'oracle.com', 'length': 335}]",05:26:00,2016-11-17,0,Asia/Tokyo
3,Chrome,Windows 8,nl-NL,175,Australia/Sydney,"[{'site': 'mail.google.com', 'length': 192}, {...",00:05:00,2016-08-23,0,Australia/Sydney
4,Firefox,Ubuntu,ro-RO,50,USA/San Francisco,"[{'site': 'mail.google.com', 'length': 266}, {...",22:55:00,2016-07-23,0,America/Los_Angeles
...,...,...,...,...,...,...,...,...,...,...
39995,Chrome,Windows 10,wo-SN,184,Malaysia/Kuala Lumpur,"[{'site': 'airbnb.com', 'length': 96}, {'site'...",06:45:00,2016-04-03,0,Asia/Kuala_Lumpur
39996,Chrome,Windows 10,ru_RU,181,France/Paris,"[{'site': 'lenta.ru', 'length': 84}, {'site': ...",20:57:00,2016-12-28,0,Europe/Paris
39997,Safari,MacOS,en-GB,112,Australia/Sydney,"[{'site': 'toptal.com', 'length': 65}, {'site'...",04:12:00,2016-07-26,0,Australia/Sydney
39998,Safari,MacOS,pt-BR,136,India/Delhi,"[{'site': 'yworks.com', 'length': 146}, {'site...",10:18:00,2017-01-01,0,Asia/Kolkata


In [ ]:
# from datetime import datetime
# from sklearn.preprocessing import OneHotEncoder
# import pandas as pd
# import pytz


# def prepare_dataframe(df: pd.DataFrame):
#     # Convert 'date' column to datetime objects
#     df['date'] = pd.to_datetime(df['date'])

#     # Convert 'time' column to datetime objects
#     df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S')  # Assuming the time format is 'hour:minute:second'

#     # Localize 'date' and 'time' columns to UTC timezone
#     df['date'] = df['date'].apply(lambda x: x if x.tzinfo else pytz.utc.localize(x))
#     df['time'] = df['time'].apply(lambda x: x if x.tzinfo else pytz.utc.localize(x))

#     df['day'] = df['date'].apply(lambda x: x.day)
#     df['month'] = df['date'].apply(lambda x: x.month)
#     df['weekday'] = df['date'].apply(lambda x: x.day_of_week)

#     df['browser'] = pd.Categorical(df['browser'])
#     df['os'] = pd.Categorical(df['os'])
#     df['locale'] = pd.Categorical(df['locale'])
#     df['location'] = pd.Categorical(df['location'])

#     df['hour'] = df['time'].apply(lambda x: x.hour)
#     df['minute'] = df['time'].apply(lambda x: x.minute)
#     df['sites_lengths'] = df['sites'].apply(lambda x: len(x))

#     df = cyclic_encode(df, 'day', 365)
#     df = cyclic_encode(df, 'month', 12)
#     df = cyclic_encode(df, 'weekday', 7)
#     df = cyclic_encode(df, 'hour', 24)
#     df = cyclic_encode(df, 'minute', 60)

#     return df.drop(columns=['sites', 'time', 'date'])

# # Assuming cyclic_encode is a function to perform cyclical encoding
# def cyclic_encode(df, column, max_value):
#     df[column + '_sin'] = np.sin(2 * np.pi * df[column] / max_value)
#     df[column + '_cos'] = np.cos(2 * np.pi * df[column] / max_value)
#     return df

# # Example usage:
# y = data['target']
# X = data.drop(columns=['target', 'user_id'])
# train_df = prepare_dataframe(X)
# train_df.head()


In [154]:
# encode cyclical features
def cyclic_encode(df, col, period):
    df[f'sin_{col}'] = np.sin(2 * np.pi * df[col].astype(int) / period)
    df[f'cos_{col}'] = np.cos(2 * np.pi * df[col].astype(int) / period)
    return df.drop(columns=[col])

In [155]:
# Correct
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder

import pandas as pd
import pytz




def prepare_dataframe(df: pd.DataFrame):

    def convert_to_local_time(row):
        location_for_utc = row['location_for_utc']
        time_str = row['time']  # Use only the time string
        input_time = datetime.strptime(time_str, '%H:%M:%S')  # Parse time only
        utc_timezone = pytz.timezone('UTC')
        local_timezone = pytz.timezone(location_for_utc)
        utc_time = utc_timezone.localize(input_time)
        local_time = utc_time.astimezone(local_timezone)
        return local_time.strftime('%H:%M:%S')

    df['local_time'] = df.apply(convert_to_local_time, axis=1)

    df['day'] = df['date'].apply(lambda x: x.day)
    df['month'] = df['date'].apply(lambda x: x.month)
    df['weekday'] = df['date'].apply(lambda x: x.day_of_week)

    df['browser'] = pd.Categorical(df['browser'])
    df['os'] = pd.Categorical(df['os'])
    df['locale'] = pd.Categorical(df['locale'])
    df['location'] = pd.Categorical(df['location'])

    df['hour'] = df['local_time'].apply(lambda x: int(str(x).split(':')[0]))
    df['minute'] = df['local_time'].apply(lambda x: int(str(x).split(':')[1]))
    # df['sites_lengths'] = df['sites'].apply(lambda x: len(x))

    df = cyclic_encode(df, 'day', 365)
    df = cyclic_encode(df, 'month', 12)
    df = cyclic_encode(df, 'weekday', 7)
    df = cyclic_encode(df, 'hour', 24)
    df = cyclic_encode(df, 'minute', 60)

    return df.drop(columns=['sites', 'time', 'date', 'location_for_utc', 'local_time'])

y = data['target']
X = data.drop(columns=['target', 'user_id'])
train_df = prepare_dataframe(X)
train_df.head()




,browser,os,locale,location,sin_day,cos_day,sin_month,cos_month,sin_weekday,cos_weekday,sin_hour,cos_hour,sin_minute,cos_minute
0,Chrome,Debian,ur-PK,Russia/Moscow,0.478734,0.877960,1.224647e-16,-1.000000e+00,0.433884,-0.900969,1.000000,6.123234e-17,-0.951057,-3.090170e-01
1,Firefox,Windows 8,uk-UA,France/Paris,0.385663,0.922640,1.000000e+00,6.123234e-17,0.974928,-0.222521,0.866025,5.000000e-01,0.587785,8.090170e-01
2,Safari,MacOS,fr-FR,Japan/Tokyo,0.288482,0.957485,-5.000000e-01,8.660254e-01,0.433884,-0.900969,-0.500000,-8.660254e-01,-1.000000,-1.836970e-16
3,Chrome,Windows 8,nl-NL,Australia/Sydney,0.385663,0.922640,-8.660254e-01,-5.000000e-01,0.781831,0.623490,0.500000,-8.660254e-01,0.866025,5.000000e-01
4,Firefox,Ubuntu,ro-RO,USA/San Francisco,0.385663,0.922640,-5.000000e-01,-8.660254e-01,-0.974928,-0.222521,-0.707107,-7.071068e-01,0.207912,9.781476e-01


In [ ]:
# # Correct
# from datetime import datetime
# from sklearn.preprocessing import OneHotEncoder

# import pandas as pd
# import pytz




# def prepare_dataframe(df: pd.DataFrame):

#     df['day'] = df['date'].apply(lambda x: x.day)
#     df['month'] = df['date'].apply(lambda x: x.month)
#     df['weekday'] = df['date'].apply(lambda x: x.day_of_week)

#     df['browser'] = pd.Categorical(df['browser'])
#     df['os'] = pd.Categorical(df['os'])
#     df['locale'] = pd.Categorical(df['locale'])
#     df['location'] = pd.Categorical(df['location'])

#     df['hour'] = df['time'].apply(lambda x: int(str(x).split(':')[0]))
#     df['minute'] = df['time'].apply(lambda x: int(str(x).split(':')[1]))
#     df['sites_lengths'] = df['sites'].apply(lambda x: len(x))

#     df = cyclic_encode(df, 'day', 365)
#     df = cyclic_encode(df, 'month', 12)
#     df = cyclic_encode(df, 'weekday', 7)
#     df = cyclic_encode(df, 'hour', 24)
#     df = cyclic_encode(df, 'minute', 60)

#     return df.drop(columns=['sites', 'time', 'date'])

# y = data['target']
# X = data.drop(columns=['target', 'user_id'])
# train_df = prepare_dataframe(X)
# train_df.head()

,browser,os,locale,location,location_for_utc,sites_lengths,sin_day,cos_day,sin_month,cos_month,sin_weekday,cos_weekday,sin_hour,cos_hour,sin_minute,cos_minute
0,Chrome,Debian,ur-PK,Russia/Moscow,Europe/Moscow,10,0.478734,0.877960,1.224647e-16,-1.000000e+00,0.433884,-0.900969,0.866025,0.500000,0.951057,0.309017
1,Firefox,Windows 8,uk-UA,France/Paris,Europe/Paris,8,0.385663,0.922640,1.000000e+00,6.123234e-17,0.974928,-0.222521,0.707107,0.707107,-0.309017,0.951057
2,Safari,MacOS,fr-FR,Japan/Tokyo,Asia/Tokyo,1,0.288482,0.957485,-5.000000e-01,8.660254e-01,0.433884,-0.900969,0.965926,0.258819,0.406737,-0.913545
3,Chrome,Windows 8,nl-NL,Australia/Sydney,Australia/Sydney,3,0.385663,0.922640,-8.660254e-01,-5.000000e-01,0.781831,0.623490,0.000000,1.000000,0.500000,0.866025
4,Firefox,Ubuntu,ro-RO,USA/San Francisco,America/Los_Angeles,8,0.385663,0.922640,-5.000000e-01,-8.660254e-01,-0.974928,-0.222521,-0.500000,0.866025,-0.500000,0.866025


In [ ]:
original_location_names = train_df['location'].unique()

In [ ]:
original_location_names

['Russia/Moscow', 'France/Paris', 'Japan/Tokyo', 'Australia/Sydney', 'USA/San Francisco', ..., 'Brazil/Rio de Janeiro', 'Singapore/Singapore', 'Netherlands/Amsterdam', 'USA/New York', 'New Zealand/Auckland']
Length: 21
Categories (21, object): ['Australia/Sydney', 'Brazil/Rio de Janeiro', 'Canada/Toronto',
                          'Canada/Vancouver', ..., 'USA/Chicago', 'USA/Miami', 'USA/New York',
                          'USA/San Francisco']

In [156]:
ohe = OneHotEncoder(sparse_output=False)

ohe_data = ohe.fit_transform(train_df[['browser','os','locale','location']])
ohe_data.shape

(40000, 56)

In [157]:
train_data = np.concatenate((ohe_data, train_df.drop(columns=['browser','os','locale','location']).values), axis=1)
train_data.shape

(40000, 66)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# pca = PCA(n_components=0.99)
# train_data = pca.fit_transform(train_data)
# train_data.shape
# scale = StandardScaler()
# train_data = scale.fit_transform(train_data)
# train_data.shape

In [158]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(train_data, y, shuffle=True, stratify=y, test_size=0.2, random_state=42)

In [ ]:
x_train

array([[ 1.        ,  0.        ,  0.        , ..., -0.70710678,
        -0.74314483,  0.66913061],
       [ 0.        ,  1.        ,  0.        , ...,  0.5       ,
         0.5       , -0.8660254 ],
       [ 1.        ,  0.        ,  0.        , ..., -0.96592583,
        -0.9945219 ,  0.10452846],
       ...,
       [ 0.        ,  1.        ,  0.        , ..., -0.70710678,
         0.10452846, -0.9945219 ],
       [ 0.        ,  1.        ,  0.        , ..., -0.5       ,
        -0.91354546, -0.40673664],
       [ 0.        ,  1.        ,  0.        , ..., -0.70710678,
        -0.30901699,  0.95105652]])

In [ ]:
# from sklearn.linear_model import LogisticRegression
# cls = LogisticRegression(max_iter=1000)

# cls.fit(x_train, y_train)

# preds = cls.predict(x_val)
# print(sklearn.metrics.classification_report(y_val, preds))

In [159]:
class ANN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, output_dim)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
x_train[0]

array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, '09:52:00', 0.3212696616923644,
       0.9469877530760753, 1.2246467991473532e-16, -1.0,
       -0.7818314824680299, 0.6234898018587334, 0.7071067811865476,
       -0.7071067811865475, -0.743144825477394, 0.6691306063588585],
      dtype=object)

In [160]:
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch_x_train = torch.tensor(x_train, dtype=torch.float32)
torch_x_val = torch.tensor(x_val, dtype=torch.float32)

torch_y_train = torch.tensor(y_train.values, dtype=torch.long)
torch_y_val = torch.tensor(y_val.values, dtype=torch.long)

train_dataset = torch.utils.data.TensorDataset(torch_x_train, torch_y_train)
val_dataset = torch.utils.data.TensorDataset(torch_x_val, torch_y_val)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2048)

In [161]:
from tqdm import tqdm
def train(model, loss_fn, optim, train_dataloader, scheduler=None):
    model.train()
    losses = []
    for inp, target in train_dataloader:
        inp = inp.to(device)
        inp.requires_grad = True
        target = target.to(device)
        optim.zero_grad()

        preds = model(inp)
        loss = loss_fn(preds, target)
        loss.backward()
        optim.step()
        losses.append(loss.item())
    return np.mean(losses)

def validation(model, val_dataloader, score_fn):
    model.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for inp, target in val_dataloader:
            inp = inp.to(device)
            target = target.to(device)
            preds = model(inp).cpu().detach().numpy().argmax(axis=1)
            y_pred.append(preds)
            y_true.append(target.cpu().detach().numpy())
    y_pred = np.hstack(y_pred)
    y_true = np.hstack(y_true)

    return score_fn(y_pred, y_true)



model = ANN(x_train.shape[-1], 2)
model.to(device)
weight=torch.tensor([0.1, 0.9], dtype=torch.float32, device=device)
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=.9)

epochs = 50
best_model_path = 'best.pth'
best_score = 0
patience = 20
pbar = tqdm(range(epochs))
for e in pbar:
    # print(f"{e + 1}/{epochs} Train loss:", train(model, loss_fn, optim, train_dataloader))
    train(model, loss_fn, optim, train_dataloader, scheduler)
    scheduler.step()
    val_score = validation(model, val_dataloader, sklearn.metrics.f1_score)
    if val_score > best_score:
        torch.save(model.state_dict(), best_model_path)
        best_score = val_score
    else:
        patience -= 1
    if patience == 0:
        break

    pbar.set_postfix({'val_score': val_score, 'best_score': best_score})

 50%|█████     | 25/50 [01:52<01:52,  4.51s/it, val_score=0.656, best_score=0.689]


In [162]:
best_model = ANN(x_train.shape[-1], 2)
best_model.to(device)
best_model.load_state_dict(torch.load(best_model_path))
best_model.eval()

ANN(
  (model): Sequential(
    (0): Linear(in_features=66, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=16, bias=True)
    (6): ReLU()
    (7): Linear(in_features=16, out_features=2, bias=True)
  )
)

In [163]:
best_model.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for inp, target in val_dataloader:
        inp = inp.to(device)
        target = target.to(device)
        preds = best_model(inp).cpu().detach().numpy().argmax(axis=1)
        y_pred.append(preds)
        y_true.append(target.cpu().detach().numpy())
y_pred = np.hstack(y_pred)
y_true = np.hstack(y_true)

print(sklearn.metrics.classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9976    1.0000    0.9988      7960
           1     1.0000    0.5250    0.6885        40

    accuracy                         0.9976      8000
   macro avg     0.9988    0.7625    0.8437      8000
weighted avg     0.9976    0.9976    0.9973      8000



In [164]:

def validation(model, val_dataloader, score_fn):
    model.eval()
    y_pred = []
    with torch.no_grad():
        for inp in val_dataloader:
            inp = inp[0]
            inp = inp.to(device)
            preds = model(inp).cpu().detach().numpy().argmax(axis=1)
            y_pred.append(preds)
    y_pred = np.hstack(y_pred)

    return y_pred

In [166]:
def get_utc_timezone(location):
    timezone_mapping = {
        'Russia/Moscow': 'Europe/Moscow',
        'France/Paris': 'Europe/Paris',
        'Japan/Tokyo': 'Asia/Tokyo',
        'Australia/Sydney': 'Australia/Sydney',
        'USA/San Francisco': 'America/Los_Angeles',
        'Canada/Vancouver': 'America/Vancouver',
        'UK/London': 'Europe/London',
        'Canada/Toronto': 'America/Toronto',
        'Malaysia/Kuala Lumpur': 'Asia/Kuala_Lumpur',
        'USA/Miami': 'America/New_York',
        'Italy/Rome': 'Europe/Rome',
        'Spain/Madrid': 'Europe/Madrid',
        'USA/Chicago': 'America/Chicago',
        'Germany/Berlin': 'Europe/Berlin',
        'China/Shanghai': 'Asia/Shanghai',
        'India/Delhi': 'Asia/Kolkata',
        'Brazil/Rio de Janeiro': 'America/Sao_Paulo',
        'Singapore/Singapore': 'Asia/Singapore',
        'Netherlands/Amsterdam': 'Europe/Amsterdam',
        'USA/New York': 'America/New_York',
        'New Zealand/Auckland': 'Pacific/Auckland'
    }
    return timezone_mapping.get(location, None)

waldo['location_for_utc'] = waldo['location'].apply(get_utc_timezone)

In [167]:
verify_df = prepare_dataframe(waldo)
ohe_data_verify = ohe.fit_transform(verify_df[['browser','os','locale','location']])
verify_data = np.concatenate((ohe_data_verify, verify_df.drop(columns=['browser','os','locale','location']).values), axis=1)



torch_verify = torch.tensor(verify_data, dtype=torch.float32)
verify_dataset = torch.utils.data.TensorDataset(torch_verify)
verify_dataloader = torch.utils.data.DataLoader(verify_dataset, batch_size=32)
test_labels = validation(best_model, verify_dataloader, best_score)


In [168]:
len([x for x in test_labels if x == 1])

108